In [12]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
import numpy as np

import sys
sys.path.append('../')

from src.bet_sizing import SimultaneousKelly

In [2]:
train_df = pd.read_csv("../data/processed/train.csv").drop(columns=["BOUT_ID", "EVENT_ID", "DATE", "BOUT_ORDINAL"])
test_df = pd.read_csv("../data/processed/test.csv").drop(columns=["BOUT_ID", "EVENT_ID", "DATE", "BOUT_ORDINAL"])

In [3]:
X_train = train_df.drop(columns=["RED_WIN"])
y_train = train_df["RED_WIN"]
X_test = test_df.drop(columns=["RED_WIN"])
y_test = test_df["RED_WIN"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LogisticRegressionCV(cv=5, max_iter=300, n_jobs=-1, random_state=0)
clf.fit(X_train_scaled, y_train)

LogisticRegressionCV(cv=5, max_iter=300, n_jobs=-1, random_state=0)

In [4]:
probs = clf.predict_proba(X_test_scaled)
red_probs, blue_probs = probs[:, 1], probs[:, 0]

In [5]:
odds = pd.read_csv("../data/processed/backtest_odds.csv")
odds["RED_MODEL_PROBS"] = red_probs
odds["BLUE_MODEL_PROBS"] = blue_probs
odds

,BOUT_ID,EVENT_ID,DATE,BOUT_ORDINAL,RED_FIGHTER_ODDS,BLUE_FIGHTER_ODDS,RED_WIN,RED_MODEL_PROBS,BLUE_MODEL_PROBS
0,3a8c0158d910546d,f5585e675af7afd4,2022-01-15,0,1.469937,2.743333,1.0,0.554607,0.445393
1,7c2a02b5ea169b54,f5585e675af7afd4,2022-01-15,1,1.261991,3.975000,1.0,0.501980,0.498020
2,3738135e4c9d8273,f5585e675af7afd4,2022-01-15,2,2.028730,1.796695,1.0,0.599743,0.400257
3,f9a4e625dffc30d4,f5585e675af7afd4,2022-01-15,6,1.462680,2.793333,1.0,0.703672,0.296328
4,8d89f3b8798f444d,f5585e675af7afd4,2022-01-15,7,1.629805,2.316667,1.0,0.569001,0.430999
...,...,...,...,...,...,...,...,...,...
910,2f414494f2b6f426,c398235fcaf8d71d,2024-03-16,8,2.920000,1.414494,0.0,0.516759,0.483241
911,acc7392d48cc09a0,c398235fcaf8d71d,2024-03-16,9,2.671429,1.480447,1.0,0.651943,0.348057
912,35f9beb94b4ef9e2,c398235fcaf8d71d,2024-03-16,10,5.448571,1.153292,1.0,0.197514,0.802486
913,a7fd9da657bbce8e,c398235fcaf8d71d,2024-03-16,11,1.466121,2.741429,NaN,0.701075,0.298925


In [6]:
event_ids = odds["EVENT_ID"].unique()   # preserves order of appearance
event_ids

array(['f5585e675af7afd4', '335ad945324c3a2e', '883922e5cd6d8473',
       'cbb682f5fcc44bfc', '77940e45bc86208e', 'f6a7b02c42e9fc5b',
       '75e0d3cc2262b732', '1f9344211ca7fd60', '1d00756835ca67c9',
       '1fac46d466abd5b8', '3a97fda0de6f1fa4', '8753e125f4499816',
       '236a37d96d476164', '6bf5a7d3e081ade2', '253d3f9e97ca149a',
       '60ba33c3f555a7a3', 'a6c32678efd4f434', '57591bbf1623574e',
       '3a24769a4855040a', 'a0a680fe2f6cc8e6', 'eb42d4febfafefd1',
       '4a9e305633f3ef47', '31da66df48c0c1a0', '8fd76e1b49c00ae2',
       '319c15b8aac5bfde', 'b0a6124751a56bc4', '8f6a18831a120817',
       'a23e63184c65f5b8', '4f853e98886283cf', '00a905a4a4a2b071',
       '93bf96be327fcd98', '8d477c3fbe001f9d', '4f670b7972fa0a2e',
       '49c29e57d4e2be6f', '885e7f70dcac0007', 'be5aab761c40ef35',
       '756f45905fb20cb5', 'b3b6e80b7d5f8f0d', '012fc7cd0779c09a',
       'b23388ff8ac6637b', 'f65a0eb902f9476b', '56ec58954158966a',
       '2e2cdb6e9eb84bb9', '5717efc6f271cd52', 'd26394fc0e8e88

In [13]:
def calculate_profit(wager_side, red_win, wager, odds):
    if np.isnan(red_win) or wager == 0:
        return 0
    
    if wager_side == "RED":
        return wager * red_win * odds - wager
    else:
        return wager * (1 - red_win) * odds - wager

In [14]:
bankroll = 100.0
for i, event_id in enumerate(event_ids):
    sliced = odds[odds["EVENT_ID"] == event_id].copy()
    kelly = SimultaneousKelly(
        red_probs=sliced["RED_MODEL_PROBS"].to_numpy(),
        blue_probs=sliced["BLUE_MODEL_PROBS"].to_numpy(),
        red_odds=sliced["RED_FIGHTER_ODDS"].to_numpy(),
        blue_odds=sliced["BLUE_FIGHTER_ODDS"].to_numpy(),
        current_bankroll=bankroll
    )

    red_wagers, blue_wagers = kelly()
    sliced["RED_WAGER"] = red_wagers
    sliced["BLUE_WAGER"] = blue_wagers

    sliced["RED_PROFIT"] = sliced.apply(lambda row: calculate_profit("RED", row["RED_WIN"], row["RED_WAGER"], row["RED_FIGHTER_ODDS"]), axis=1).round(2)
    sliced["BLUE_PROFIT"] = sliced.apply(lambda row: calculate_profit("BLUE", row["RED_WIN"], row["BLUE_WAGER"], row["BLUE_FIGHTER_ODDS"]), axis=1).round(2)

    bankroll += sliced["RED_PROFIT"].sum() + sliced["BLUE_PROFIT"].sum()
    bankroll = round(bankroll, 2)
    print(f"Event {i + 1}/{len(event_ids)}: Bankroll = {bankroll}")

Event 1/94: Bankroll = 99.68
Event 2/94: Bankroll = 96.4
Event 3/94: Bankroll = 92.04
Event 4/94: Bankroll = 84.45
Event 5/94: Bankroll = 85.12
Event 6/94: Bankroll = 83.11
Event 7/94: Bankroll = 80.52
Event 8/94: Bankroll = 76.56
Event 9/94: Bankroll = 77.94
Event 10/94: Bankroll = 84.62
Event 11/94: Bankroll = 87.41
Event 12/94: Bankroll = 84.86
Event 13/94: Bankroll = 84.62
Event 14/94: Bankroll = 89.3
Event 15/94: Bankroll = 93.65
Event 16/94: Bankroll = 100.17
Event 17/94: Bankroll = 101.21
Event 18/94: Bankroll = 100.11
Event 19/94: Bankroll = 93.98
Event 20/94: Bankroll = 103.12
Event 21/94: Bankroll = 108.1
Event 22/94: Bankroll = 100.41
Event 23/94: Bankroll = 103.93
Event 24/94: Bankroll = 110.0
Event 25/94: Bankroll = 114.77
Event 26/94: Bankroll = 119.51
Event 27/94: Bankroll = 115.61
Event 28/94: Bankroll = 127.75
Event 29/94: Bankroll = 136.6
Event 30/94: Bankroll = 130.12
Event 31/94: Bankroll = 131.96
Event 32/94: Bankroll = 128.09
Event 33/94: Bankroll = 128.08
Event 3

c:\Users\eugen\Documents\school\college\spring 2024\CPSC 381\cpsc-381-final-project\.venv\lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Event 79/94: Bankroll = 257.44
Event 80/94: Bankroll = 259.47
Event 81/94: Bankroll = 236.57
Event 82/94: Bankroll = 252.23
Event 83/94: Bankroll = 260.74
Event 84/94: Bankroll = 264.97
Event 85/94: Bankroll = 283.34
Event 86/94: Bankroll = 267.57
Event 87/94: Bankroll = 297.3
Event 88/94: Bankroll = 320.05
Event 89/94: Bankroll = 330.04
Event 90/94: Bankroll = 312.46
Event 91/94: Bankroll = 282.2
Event 92/94: Bankroll = 292.61
Event 93/94: Bankroll = 317.45
Event 94/94: Bankroll = 314.48
